# 제주 한달살기 숙소 추천을 위한 분석 - 크롤링


### 미스터멘션 - https://www.mrmention.co.kr/
### 한달살기 전문사이트
- 이유 : 한달살기를 위한 시설이 준비된 숙소
- case 155건

In [ ]:
# # 패키지 list
# # !pip freeze > requirements.txt

# package_list = open('./requirements.txt', 'r')
# package_list = package_list.read()
# print(package_list)

In [1]:
# !pip install selenium pandas bs4 xlrd requests tqdm pyautogui

import pandas as pd
import numpy as np

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

from bs4 import BeautifulSoup
import requests


from urllib.parse import quote

import time
#time.sleep을 위한 난수생성
import random

from tqdm import tqdm

# text 전처리
import re

# 마우스 커서 이동을 위한 패키지
import pyautogui

## 미스터멘션 
메뉴 - 한달살이 best <br>
지역 검색 - 제주 <br>
https://www.mrmention.co.kr/v2/search?theme=31&title=%EC%A0%9C%EC%A3%BC&as_path=0

In [20]:
# step 1 - 숙소 기본정보 크롤링

url = 'https://www.mrmention.co.kr/v2/search?theme=31&title=%EC%A0%9C%EC%A3%BC&as_path=0'

driver = webdriver.Chrome()
driver.get(url)
driver.implicitly_wait(10)
driver.maximize_window()
time.sleep(2.4)

# 지도창 닫기
driver.find_element(By.CLASS_NAME, 'search_map_toggle_btn__GAhJd').click()

# 영역 선택이 어려워서 마우스 커서를 해당영역으로 이동시키는 
pyautogui.moveTo(465 , 800 , 2)	
pyautogui.leftClick()

# 스크롤을 내려서 모든 숙소 확인
for _ in range(30):
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)  
    time.sleep(round(random.uniform(0.3, 1.2), 3))
time.sleep(round(random.uniform(1.3, 2.2), 3))

# 크롤링 
soup = BeautifulSoup(driver.page_source, 'html.parser')
accom_list =soup.select('.jsx-5f908135cc97e6c3 > .AccommodationCard_room_info_con__hyAro')

temp = []
for accom in accom_list:
    title = accom.select_one('.AccommodationCard_room_tit__f6lnu').text.strip()
    day = accom.select_one('.AccommodationCard_amount__dyci8').select('.Price_day__SUxE_')[-1].text.strip()
    price = accom.select_one('.AccommodationCard_amount__dyci8').select('.Price_price__mdIcc')[-1].text.strip()
    price = re.sub('[^0-9]','',price)
    rating = accom.select_one('.AccommodationCard_rating_sco__75Yd9').text.strip()
    reviewNum = accom.select_one('.AccommodationCard_rating_count__fesL7').text.strip()[1:-1]
    location = accom.select_one('.AccommodationCard_area___0RDd').text.strip()

    temp.append({'title' : title,
                 'day' : day,
                 'price' : price,
                 'rating' : rating,
                 'reviewNum' : reviewNum,
                 'location' : location})
    
temp = pd.DataFrame(temp)
temp.to_csv('./data/mmention_20240112.csv', index=False)

In [21]:
temp

,title,day,price,rating,reviewNum,location
0,[서귀포/안덕] #프리미엄 스테이 #중문관광단지 인접 #소파 #엔티크 인테리어 알리...,29박,1193060,4.5,82,안덕면
1,[친환경 숙소] [제주/애월] #최대18명 #방5개 부엌2개 #워크샵 #대가족 #단...,29박,4466000,4.7,24,애월읍
2,[제주/제주시] #신축 레지던스 #제주시내 초근접 #공항10분거리 #취사가능 : ...,29박,899962,4.8,245,노형동
3,[제주/한림] #애견동반 가능 #논밭뷰 #복층다락방 '스테이 앤틱',29박,2030020,4.7,18,한림읍
4,[제주시] 제주공항 근처 나만 알고 싶은 포근한 대가족 펜션 / 키즈 펜션,29박,1039940,5,75,신비마을1길
...,...,...,...,...,...,...
149,[제주/노형] 제주 도심에서의 생활 #가성비 #제주시내,29박,2871000,0,0,노형동
150,[제주/애월] 유티에스빌리지,29박,1229872,4,45,애월읍
151,[친환경 숙소] [제주/한림] #단독 독채 #테라스 #해먹 #논밭뷰 #옥상단상 '설레임',29박,2399996,4.5,13,한림읍
152,[제주시] 애월바다를 품은 곽지해수욕장 신축펜션 팀버하우스,29박,2073500,4.9,46,애월읍


In [24]:
temp['day'].unique()

array(['29박'], dtype=object)

숙소개별 상세정보/리뷰 크롤링

In [8]:
# step 1 - 숙소 기본정보 크롤링

url = 'https://www.mrmention.co.kr/v2/search?theme=31&title=%EC%A0%9C%EC%A3%BC&as_path=0'

driver = webdriver.Chrome()
driver.get(url)
driver.implicitly_wait(10)
driver.maximize_window()
time.sleep(2.4)

# 지도창 닫기
driver.find_element(By.CLASS_NAME, 'search_map_toggle_btn__GAhJd').click()

# 영역 선택이 어려워서 마우스 커서를 해당영역으로 이동시키는 
pyautogui.moveTo(465 , 800 , 2)	
pyautogui.leftClick()

# 스크롤을 내려서 모든 숙소 확인
for _ in range(30):
    driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)  
    time.sleep(round(random.uniform(0.3, 1.2), 3))
time.sleep(round(random.uniform(1.3, 2.2), 3))

# 크롤링 
soup = BeautifulSoup(driver.page_source, 'html.parser')
accom_list =soup.select('.jsx-5f908135cc97e6c3 > .AccommodationCard_room_info_con__hyAro')

temp = []
for accom in accom_list:
    title = accom.select_one('.AccommodationCard_room_tit__f6lnu').text.strip()
    day = accom.select_one('.AccommodationCard_amount__dyci8').select('.Price_day__SUxE_')[-1].text.strip()
    price = accom.select_one('.AccommodationCard_amount__dyci8').select('.Price_price__mdIcc')[-1].text.strip()
    price = re.sub('[^0-9]','',price)
    rating = accom.select_one('.AccommodationCard_rating_sco__75Yd9').text.strip()
    reviewNum = accom.select_one('.AccommodationCard_rating_count__fesL7').text.strip()[1:-1]
    location = accom.select_one('.AccommodationCard_area___0RDd').text.strip()

    temp.append({'title' : title,
                 'day' : day,
                 'price' : price,
                 'rating' : rating,
                 'reviewNum' : reviewNum,
                 'location' : location})
    
temp = pd.DataFrame(temp)
temp.to_csv('./data/mmention_20240115.csv', index=False)

### 상세정보

pension_list = soup.select_one('.search_search_list__Zfhwh').select('a')
review_result = []
detail_result = []

for pension in pension_list:
    sub_url = pension.attrs['href']
    base_url = 'https://www.mrmention.co.kr/'
    url = base_url + sub_url
    driver.get(url)

    driver.maximize_window()
    driver.implicitly_wait(10)

    
    
    ### 구비시설 
    driver.find_element(By.CLASS_NAME, 'Amenity_ame_more__rOwp1').click()
    time.sleep(round(random.uniform(1.0, 1.5), 3))

    ammen = driver.find_element(By.CLASS_NAME, 'AmenityPopup_ame_pop_con__NhQad').text.strip().replace('\n', ' ')
    
    driver.find_element(By.CLASS_NAME, 'AmenityPopup_close__8jpjy').click()
    time.sleep(round(random.uniform(1.0, 1.5), 3))

    ### review

    # 스크롤을 끝까지 내려서 모든 정보 획득
    # 크롤링 방지를 피하기 위해 한번에 스크롤 내리지 않고, 속임수를 사용
    while True:
        last_height = driver.execute_script("return document.body.scrollHeight")

        # 창의 내리기 끝까지 하지않고, 중간에 멈춰서 한번은 page_down을 통해서
        driver.execute_script(f'window.scrollTo(0, {last_height - random.randint(1, 100)})')
        time.sleep(round(random.uniform(1.0, 1.5), 3))
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.PAGE_DOWN)     
        time.sleep(round(random.uniform(0.3, 1.2), 3))

        # 창의 높이가 달라지지 않으면 스크롤 내리기 중단
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            break

    # 리뷰 더보기 버튼 클릭
    driver.find_elements(By.CLASS_NAME, 'ToggleBtn_accom_toggle_btn_blur__pg8Mg')[1].click()
    time.sleep(round(random.uniform(1.3, 2.2), 3))

    mini_soup = BeautifulSoup(driver.page_source, 'html.parser')

    title = mini_soup.select_one('.AccommodationTitle_title__kNF2R').text
    review_list = mini_soup.select_one('#Review_review__5RC6b').select('li')
    
    #리뷰가 없는 경우 pass
    try :
        for review in review_list:
            review_text = review.select_one('.Review_is_review__txxuK').text
            review_date = review.select_one('.Review_date__yjft1').text
            review_star = review.select_one('.Star_frontStarDiv__xU1lg').get_attribute_list('style')[0]
            review_star = re.sub('[^0-9]', '',review_star)
            review_star = int(review_star) / 20
            review_result.append([title, review_text, review_date, review_star, ammen])
    except :
        pass

review_result = pd.DataFrame(review_result)
review_result.columns = ['title', 'review', 'review_date', 'star', 'ammen']
review_result.to_csv('./data/mmention_review_20240109.csv', index=False)

In [12]:
review_result = pd.DataFrame(review_result)
review_result.columns = ['title', 'review', 'review_date', 'star', 'ammen']

In [13]:
review_result

,title,review,review_date,star,ammen
0,[제주/한림] #애견동반 가능 #논밭뷰 #복층다락방 '스테이 앤틱',한달살기 하면서 날씨도 좋고 모든게 좋았습니다~~단지내에 계시던 주민모두 너무 친절...,2022년 12월,20.0,생활시설 주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락...
1,[제주/한림] #애견동반 가능 #논밭뷰 #복층다락방 '스테이 앤틱',"일년을 고민 끝에 떠난 제주도,,,\n자는 곳이 제일 중해서 고심 끝에 결정했습니다...",2022년 12월,20.0,생활시설 주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락...
2,[제주/한림] #애견동반 가능 #논밭뷰 #복층다락방 '스테이 앤틱',퇴사하고 퇴직금 털어서 부모님 모시고 다녀왔습니다.\n독채는 신경쓸게 많을 것 같아...,2022년 12월,20.0,생활시설 주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락...
3,[제주/한림] #애견동반 가능 #논밭뷰 #복층다락방 '스테이 앤틱',9/1부터 2주간 제주에서 지내기 위해서 정한 스테이입니다. 큰도로에서 좀 들어와 ...,2022년 9월,20.0,생활시설 주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락...
4,[제주/한림] #애견동반 가능 #논밭뷰 #복층다락방 '스테이 앤틱',4층이지만 엘리베이터가 있어서 전혀불편함 없이 생활했어요. 스테이상태가 너무좋아서 ...,2022년 6월,20.0,생활시설 주차장 엘리베이터 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락...
...,...,...,...,...,...
3675,[제주시] 신축풀옵숙소 한달살기 '라온스테이',사실과다른내용이면 고지없이 삭제한다니 마음에들지 않으면 지우겠다는거네요 알아서 ...,2020년 11월,20.0,생활시설 주차장 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 세면도구 ...
3676,[제주시] 신축풀옵숙소 한달살기 '라온스테이',시외버스터미널과 가까워서 제주 어디로든 대중교통을 이용해 이동하기 편해요. 방도 두...,2022년 11월,16.0,생활시설 주차장 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 세면도구 ...
3677,[제주시] 신축풀옵숙소 한달살기 '라온스테이',일반 원투룸형 숙소로 공항에서 가깝고 주차도 편리하다.. 주택가라 조용해서 좋았는데...,2022년 9월,16.0,생활시설 주차장 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 세면도구 ...
3678,[제주시] 신축풀옵숙소 한달살기 '라온스테이',공항에서 택시비 5000원도 안나와요 \r\n도착첫날이나 마지막날 이용하기 좋아요\...,2022년 6월,16.0,생활시설 주차장 TV 케이블 TV 에어컨 와이파이 세탁기 (개인) 도어락 세면도구 ...


In [3]:
df = pd.read_csv('./data/mmention_20240112.csv')
df


,title,day,price,rating,reviewNum,location
0,[서귀포/안덕] #프리미엄 스테이 #중문관광단지 인접 #소파 #엔티크 인테리어 알리...,29박,1193060,4.5,82,안덕면
1,[친환경 숙소] [제주/애월] #최대18명 #방5개 부엌2개 #워크샵 #대가족 #단...,29박,4466000,4.7,24,애월읍
2,[제주/제주시] #신축 레지던스 #제주시내 초근접 #공항10분거리 #취사가능 : ...,29박,899962,4.8,245,노형동
3,[제주/한림] #애견동반 가능 #논밭뷰 #복층다락방 '스테이 앤틱',29박,2030020,4.7,18,한림읍
4,[제주시] 제주공항 근처 나만 알고 싶은 포근한 대가족 펜션 / 키즈 펜션,29박,1039940,5.0,75,신비마을1길
...,...,...,...,...,...,...
149,[제주/노형] 제주 도심에서의 생활 #가성비 #제주시내,29박,2871000,0.0,0,노형동
150,[제주/애월] 유티에스빌리지,29박,1229872,4.0,45,애월읍
151,[친환경 숙소] [제주/한림] #단독 독채 #테라스 #해먹 #논밭뷰 #옥상단상 '설레임',29박,2399996,4.5,13,한림읍
152,[제주시] 애월바다를 품은 곽지해수욕장 신축펜션 팀버하우스,29박,2073500,4.9,46,애월읍


In [4]:
df['title'][0].split(' ')[-2] + df['title'][0].split(' ')[-1]

'인제주'

In [2]:
base_url = 'https://search.naver.com/search.naver?where=nexearch&sm=tab_jum&query='

addr_list = []
for pension in tqdm(df['title']):
    # for name in pension.split(' '):
    #     print(name)

    name = pension.split(' ')[-1]
    keyword_list = ['제주', name]
    keyword_search = ''

    for word in keyword_list:    
        keyword_search += f'+{quote(word)}'

    # 검색 URL - 네이버 VIEW 중에서 BLOG / 숙소이름 / 제외검색어
    url = base_url + keyword_search    
    options = webdriver.ChromeOptions()
    # 창 숨기는 옵션 추가
    options.add_argument("headless")
    driver = webdriver.Chrome(options=options)
    driver.get(url)

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    try :
        addr1 = soup.select_one('.place_section_content').select_one('.LDgIH').text.strip()        
    except :
        addr1 = 'none'

    name = pension.split(' ')[-2] + pension.split(' ')[-1]
    keyword_list = ['제주', name]
    keyword_search = ''

    for word in keyword_list:    
        keyword_search += f'+{quote(word)}'

    # 검색 URL - 네이버 VIEW 중에서 BLOG / 숙소이름 / 제외검색어
    url = base_url + keyword_search    
    options = webdriver.ChromeOptions()
    # 창 숨기는 옵션 추가
    options.add_argument("headless")
    driver = webdriver.Chrome(options=options)
    driver.get(url)

    soup = BeautifulSoup(driver.page_source, 'html.parser')

    try :
        addr2 = soup.select_one('.place_section_content').select_one('.LDgIH').text.strip()        
    except :
        addr2 = 'none'
    
    addr_list.append([pension, name, addr1, addr2])

    time.sleep(1)
    
df_addr = pd.DataFrame(addr_list)
df_addr    
    



NameError: name 'df' is not defined